In [11]:
# dependencies
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [12]:
# read cancercsv into dataframe
cancer_df = pd.read_csv("csv/skin_map_incidence.csv")
cancer_df.head()

,State,Range,Rate
0,AK,9.5 to 19.0,14.9
1,AL,9.5 to 19.0,19.0
2,AR,19.1 to 21.9,20.0
3,AZ,19.1 to 21.9,20.6
4,CA,23.0 to 25.8,23.0


In [13]:
# read solar install csv into dataframe
solar_df = pd.read_csv("csv/openpv-export-201803141836.csv")
solar_df.head(2)

,state,date_installed,incentive_prog_names,type,size_kw,appraised,zipcode,install_type,installer,cost_per_watt,...,effic_1,cust_county,tracking,inv_model_clean,mod_cost_real,inv_cost_real,bos_powerclerk_real,permitting_real,3rdparty,Filters=maxdate=1420045199&mindate=1388552400
0,CO,01/01/2014,NaN,uir,13.58,NaN,80401,Government,NaN,1.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NY,01/01/2014,NaN,uir,306.60,NaN,11215,commercial,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
solar_targs_df = solar_df.iloc[:,[0, 1, 4, 6, 9]]
solar_targs_df.columns = ["State", "Install_Data", "Size_kw", "Zip", "Cost_per_Watt"]
solar_targs_df.head(2)

,State,Install_Data,Size_kw,Zip,Cost_per_Watt
0,CO,01/01/2014,13.58,80401,1.62
1,NY,01/01/2014,306.60,11215,NaN


In [15]:
# check to see how deeply the nans permiate our data
solar_targs_df.count()

State            170862
Install_Data     170862
Size_kw          170862
Zip              170862
Cost_per_Watt     73212
dtype: int64

In [16]:
# read eia cost data into dataframe
eia_df = pd.read_csv("csv/eia_2014_scrape.csv")
eia_df = eia_df.iloc[:,[1,2]]
eia_df["State"] = ["AL","AK","AZ","AR","CA","CO","CT","DE","DC","FL","GA","HI","ID","IL","IN","IA","KS",
                   "KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC",
                   "ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY"]
#eia_df

In [17]:
# merge cancer and electric cost dfs
cande_df = pd.merge(cancer_df, eia_df, on="State", how="outer")
#cande_df

In [18]:
# merge solar and cande dfs
data_df = pd.merge(cande_df, solar_targs_df, on="State", how="outer")

# reset index to serve as id column
data_df = data_df.reset_index()
#data_df.head()

# rename columns and reorder columns
data_df.columns = ["Id", "Abbr", "Range", "Cancer_Rate", "State", "Electric_Cost (cents/kWh)", "Install_Date",
                  "Size_kW", "Zipcode", "Cost_per_Watt"]
data_df = data_df[["Id", "State", "Abbr", "Electric_Cost (cents/kWh)", "Install_Date", "Zipcode", "Size_kW",
                 "Cost_per_Watt", "Range", "Cancer_Rate"]]
#data_df.head(10)

# push clean df to csv
data_df.to_csv('csv/clean_data.csv')

In [19]:
# push data to sqlite db
engine = create_engine('sqlite:///clean_solar_data.db')
data_df.to_sql("data", con=engine, flavor="sqlite", if_exists="replace", index=False)

C:\Users\brbal\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\pandas\io\sql.py:527: FutureWarning: the 'flavor' parameter is deprecated and will be removed in a future version, as 'sqlite' is the only supported option when SQLAlchemy is not installed.
  _validate_flavor_parameter(flavor)


In [21]:
df = pd.read_sql_query('SELECT * FROM "data" LIMIT 3', engine)
df.head()

,Id,State,Abbr,Electric_Cost (cents/kWh),Install_Date,Zipcode,Size_kW,Cost_per_Watt,Range,Cancer_Rate
0,0,Alaska,AK,17.46,03/05/2014,99712.0,2.5,3.2,9.5 to 19.0,14.9
1,1,Alaska,AK,17.46,03/12/2014,99701.0,12.0,NaN,9.5 to 19.0,14.9
2,2,Alabama,AL,9.27,None,NaN,NaN,NaN,9.5 to 19.0,19.0
